In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import os

In [2]:
articles = pd.read_csv("data/articles.csv")
articles.head(2)

,article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,...,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc
0,108775015,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,9,Black,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
1,108775044,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,10,White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.


In [3]:
df = articles
df.shape

(105542, 25)

In [4]:
df.head(2)

,article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,...,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc
0,108775015,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,9,Black,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
1,108775044,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,10,White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.


In [5]:
df.columns

Index(['article_id', 'product_code', 'prod_name', 'product_type_no',
       'product_type_name', 'product_group_name', 'graphical_appearance_no',
       'graphical_appearance_name', 'colour_group_code', 'colour_group_name',
       'perceived_colour_value_id', 'perceived_colour_value_name',
       'perceived_colour_master_id', 'perceived_colour_master_name',
       'department_no', 'department_name', 'index_code', 'index_name',
       'index_group_no', 'index_group_name', 'section_no', 'section_name',
       'garment_group_no', 'garment_group_name', 'detail_desc'],
      dtype='object')

In [6]:
df = df[["article_id", #"product_code", 
         "product_type_no", "graphical_appearance_no",
         #"colour_group_code", "perceived_colour_value_id", "perceived_colour_master_id",
         "department_no", "index_group_no", "section_no", "garment_group_no"]]

df.sample(5)

,article_id,product_type_no,graphical_appearance_no,department_no,index_group_no,section_no,garment_group_no
5035,488697029,94,1010028,3629,3,27,1020
57968,718262001,255,1010005,8748,4,47,1005
14365,562498002,272,1010016,1676,1,16,1002
75329,780947004,87,1010016,6561,4,42,1020
63930,739903002,245,1010016,7618,4,76,1003


In [7]:
df.isnull().sum()

article_id                 0
product_type_no            0
graphical_appearance_no    0
department_no              0
index_group_no             0
section_no                 0
garment_group_no           0
dtype: int64

In [8]:
def convertToList(x):
    return [x]

In [9]:
columns_to_transform = [
         #"product_code", 
         "product_type_no", "graphical_appearance_no",
         #"colour_group_code", "perceived_colour_value_id", "perceived_colour_master_id",
         "department_no", "index_group_no", "section_no", "garment_group_no"
]

# df.loc[:, columns_to_transform] = df[columns_to_transform].apply(lambda col: col.apply(convertToList))

In [10]:
df.head(2)

,article_id,product_type_no,graphical_appearance_no,department_no,index_group_no,section_no,garment_group_no
0,108775015,253,1010016,1676,1,16,1002
1,108775044,253,1010016,1676,1,16,1002


In [11]:
columns_to_aggregrate = [
         #"product_code", 
         "product_type_no", "graphical_appearance_no",
         #"colour_group_code", "perceived_colour_value_id", "perceived_colour_master_id",
         "department_no", "index_group_no", "section_no", "garment_group_no"
]

train = pd.DataFrame(df['article_id'])

train['features'] = df[columns_to_aggregrate].values.tolist()

In [12]:
train.head(2)

,article_id,features
0,108775015,"[253, 1010016, 1676, 1, 16, 1002]"
1,108775044,"[253, 1010016, 1676, 1, 16, 1002]"


In [13]:
vector = train['features'].values.tolist()
vector = np.array(vector)
vector.shape

(105542, 6)

In [14]:
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity

In [15]:
n_components = 6
svd = TruncatedSVD(n_components=n_components)
from sklearn.decomposition import PCA

# ## for now feeding 10000 rows only
# reduced_vector = svd.fit_transform(vector)

n_components = 3
pca = PCA(n_components=n_components)
reduced_vector = pca.fit_transform(vector)

batch_size = 10000 
similarity_matrix = []

for i in range(0, reduced_vector.shape[0], batch_size):
    batch = reduced_vector[i:i + batch_size]
    similarity_batch = cosine_similarity(batch, reduced_vector)
    similarity_matrix.append(similarity_batch)
    print(i+batch_size)

similarity_matrix = np.vstack(similarity_matrix)

10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000


In [17]:
similarity_matrix

array([[1.        , 1.        , 0.99999994, ..., 0.99999015, 0.84116925,
        0.99999015],
       [1.        , 1.        , 0.99999994, ..., 0.99999015, 0.84116925,
        0.99999015],
       [0.99999994, 0.99999994, 1.        , ..., 0.99998939, 0.84133811,
        0.99998939],
       ...,
       [0.99999015, 0.99999015, 0.99998939, ..., 1.        , 0.83930057,
        1.        ],
       [0.84116925, 0.84116925, 0.84133811, ..., 0.83930057, 1.        ,
        0.83930057],
       [0.99999015, 0.99999015, 0.99998939, ..., 1.        , 0.83930057,
        1.        ]])

In [18]:
from joblib import dump

dump(similarity_matrix, 'artifacts/similarity_matrix.pkl')

['artifacts/similarity_matrix.pkl']

In [24]:
dump(articles, 'artifacts/articles.pkl')

['artifacts/articles.pkl']